In [1]:
import requests
import zipfile
import os
import shutil

# URL for the dataset
url = 'https://www.kaggle.com/api/v1/datasets/download/ikrambenabd/malimg-original'

# Download the file
print("Downloading the dataset...")
response = requests.get(url)

import time

# Remove the extraction directory if it exists to avoid permission errors
extract_dir = 'malimg_paper_dataset_imgs'
if os.path.exists(extract_dir):
    print(f"Removing existing directory: {extract_dir}")
    shutil.rmtree(extract_dir)
    # Wait until the directory is actually removed
    while os.path.exists(extract_dir):
        time.sleep(0.1)

# Check if download was successful
if response.status_code == 200:
    # Save the file
    with open('malimg.zip', 'wb') as f:
        f.write(response.content)
    print("Download completed!")
    
    # Extract the zip file
    print("Extracting files...")
    with zipfile.ZipFile('malimg.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    print("Extraction completed!")
else:
    print(f"Failed to download. Status code: {response.status_code}")
    print(f"Response: {response.text[:500]}...")  # Print first 500 chars of response

Removing existing directory: malimg_paper_dataset_imgs
Download completed!
Extracting files...


PermissionError: [Errno 13] Permission denied: 'malimg_paper_dataset_imgs\\Allaple.A\\040c4d328f57614e1dd2d8c5e00b0d55.png'

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

DATA_BASE_PATH = "./malimg_paper_dataset_imgs/"

KeyboardInterrupt: 

In [ ]:
# compute the class distribution
dist = {}
for mlw_class in os.listdir(DATA_BASE_PATH):
    mlw_dir = os.path.join(DATA_BASE_PATH, mlw_class)
    dist[mlw_class] = len(os.listdir(mlw_dir))

In [ ]:
# plot the class distribution

# HTB Color Palette
htb_green = "#9FEF00"
node_black = "#141D2B"
hacker_grey = "#A4B1CD"

# data
classes = list(dist.keys())
frequencies = list(dist.values())

# plot
plt.figure(facecolor=node_black)
sns.barplot(y=classes, x=frequencies, edgecolor = "black", orient='h', color=htb_green)
plt.title("Malware Class Distribution", color=htb_green)
plt.xlabel("Malware Class Frequency", color=htb_green)
plt.ylabel("Malware Class", color=htb_green)
plt.xticks(color=hacker_grey)
plt.yticks(color=hacker_grey)
ax = plt.gca()
ax.set_facecolor(node_black)
ax.spines['bottom'].set_color(hacker_grey)
ax.spines['top'].set_color(node_black)
ax.spines['right'].set_color(node_black)
ax.spines['left'].set_color(hacker_grey)
plt.show()

In [ ]:
pip install split-folders

In [ ]:
import splitfolders

DATA_BASE_PATH = "./malimg_paper_dataset_imgs/"
TARGET_BASE_PATH = "./newdata/"

TRAINING_RATIO = 0.8
TEST_RATIO = 1 - TRAINING_RATIO

splitfolders.ratio(input=DATA_BASE_PATH, output=TARGET_BASE_PATH, ratio=(TRAINING_RATIO, 0, TEST_RATIO))

## Handling PermissionError and File Path Issues in Python

This section demonstrates best practices for handling file paths and permissions in Python, especially on Windows. It includes examples for writing to files, handling PermissionError, and modifying file permissions if needed.

In [ ]:
import os

# Example: Proper file path handling and permission modification
file_path = r"C:\Users\R.Daswanta kumar\OneDrive\Pictures\GFG\file\GFG.txt"

try:
    if os.path.exists(file_path):
        # Attempt to modify file permissions to allow writing
        os.chmod(file_path, 0o666)
        print("File permissions modified successfully!")
    else:
        print("File not found:", file_path)
except PermissionError:
    print("Permission denied: You don't have the necessary permissions to change the permissions of this file.")

try:
    with open(file_path, "w") as file:
        new_content = "New content to replace the existing content."
        file.write(new_content)
    print("File content modified successfully!")
except PermissionError:
    print("Permission denied: You don't have the necessary permissions to write to this file.")

---

**Key Points:**
- Always use raw strings (prefix with `r`) or double backslashes for Windows file paths.
- Use `os.chmod` to attempt to change file permissions if you encounter a PermissionError.
- Always handle exceptions to provide clear error messages and avoid crashes.